In [2]:
# Import af pakker
import pandas as pd
import spacy
import re
from danlp.models import load_bert_emotion_model

# Indlæs dansk sprogmodel
nlp = spacy.load("da_core_news_sm") # Indlæser sprogmodel

# Indlæs emotion classifier
classifier = load_bert_emotion_model()

# Indlæs data
redditdata_url = "https://raw.githubusercontent.com/CALDISS-AAU/course_ndms-I/master/datasets/reddit_rdenmark-comments_01032021-08032021_long.csv"
reddit_df = pd.read_csv(redditdata_url)

/opt/tljh/user/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
reddit_df.head()

,post_author,post_created_utc,post_domain,post_full_link,post_gildings,post_id,post_is_original_content,post_is_reddit_media_domain,post_locked,post_media_only,...,comment_is_submitter,comment_link_id,comment_parent_id,comment_permalink,comment_retrieved_on,comment_score,comment_stickied,comment_subreddit,comment_subreddit_id,comment_total_awards_received
0,MadsenFraMadsenOgCo,1614698948,berlingske.dk,https://www.reddit.com/r/Denmark/comments/lw4k...,{},lw4koj,False,False,False,False,...,False,t3_lw4koj,t3_lw4koj,/r/Denmark/comments/lw4koj/pfizer_med_opjuster...,1614699643,1,False,Denmark,t5_2qjto,0
1,MadsenFraMadsenOgCo,1614698948,berlingske.dk,https://www.reddit.com/r/Denmark/comments/lw4k...,{},lw4koj,False,False,False,False,...,False,t3_lw4koj,t1_gpfa37s,/r/Denmark/comments/lw4koj/pfizer_med_opjuster...,1614700857,1,False,Denmark,t5_2qjto,0
2,MadsenFraMadsenOgCo,1614698948,berlingske.dk,https://www.reddit.com/r/Denmark/comments/lw4k...,{},lw4koj,False,False,False,False,...,False,t3_lw4koj,t1_gpfcfj3,/r/Denmark/comments/lw4koj/pfizer_med_opjuster...,1614702086,1,False,Denmark,t5_2qjto,0
3,MadsenFraMadsenOgCo,1614698948,berlingske.dk,https://www.reddit.com/r/Denmark/comments/lw4k...,{},lw4koj,False,False,False,False,...,False,t3_lw4koj,t1_gpfesm5,/r/Denmark/comments/lw4koj/pfizer_med_opjuster...,1614702455,1,False,Denmark,t5_2qjto,0
4,MadsenFraMadsenOgCo,1614698948,berlingske.dk,https://www.reddit.com/r/Denmark/comments/lw4k...,{},lw4koj,False,False,False,False,...,False,t3_lw4koj,t1_gpffidu,/r/Denmark/comments/lw4koj/pfizer_med_opjuster...,1614702670,1,False,Denmark,t5_2qjto,0


In [4]:
# Find et opslag med tekstindhold fra Reddit datasættet, der har mere end 100 kommentarer.

reddit_df.loc[:, ['post_id', 'post_num_comments']]

,post_id,post_num_comments
0,lw4koj,17
1,lw4koj,17
2,lw4koj,17
3,lw4koj,17
4,lw4koj,17
...,...,...
3423,ly94u0,23
3424,ly94u0,23
3425,ly94u0,23
3426,ly94u0,23


In [5]:
reddit_posts = reddit_df.drop_duplicates(subset = ['post_id'])
reddit_posts.head()

,post_author,post_created_utc,post_domain,post_full_link,post_gildings,post_id,post_is_original_content,post_is_reddit_media_domain,post_locked,post_media_only,...,comment_is_submitter,comment_link_id,comment_parent_id,comment_permalink,comment_retrieved_on,comment_score,comment_stickied,comment_subreddit,comment_subreddit_id,comment_total_awards_received
0,MadsenFraMadsenOgCo,1614698948,berlingske.dk,https://www.reddit.com/r/Denmark/comments/lw4k...,{},lw4koj,False,False,False,False,...,False,t3_lw4koj,t3_lw4koj,/r/Denmark/comments/lw4koj/pfizer_med_opjuster...,1614699643,1,False,Denmark,t5_2qjto,0
17,Jell929,1614702056,self.Denmark,https://www.reddit.com/r/Denmark/comments/lw5u...,{},lw5uiy,False,False,False,False,...,False,t3_lw5uiy,t3_lw5uiy,/r/Denmark/comments/lw5uiy/nogen_der_spiller_s...,1614705435,1,False,Denmark,t5_2qjto,0
34,HitmanZeus,1614704335,bt.dk,https://www.reddit.com/r/Denmark/comments/lw6s...,{},lw6sew,False,False,False,False,...,False,t3_lw6sew,t3_lw6sew,/r/Denmark/comments/lw6sew/københavns_universi...,1614705758,1,False,Denmark,t5_2qjto,0
71,have_nisse,1614704603,berlingske.dk,https://www.reddit.com/r/Denmark/comments/lw6w...,{},lw6wtr,False,False,False,False,...,False,t3_lw6wtr,t3_lw6wtr,/r/Denmark/comments/lw6wtr/mette_frederiksens_...,1614706927,1,False,Denmark,t5_2qjto,0
97,mariayten,1614706848,self.Denmark,https://www.reddit.com/r/Denmark/comments/lw7u...,{},lw7u7u,False,False,False,False,...,False,t3_lw7u7u,t3_lw7u7u,/r/Denmark/comments/lw7u7u/points_of_interest_...,1614709239,1,False,Denmark,t5_2qjto,0


In [15]:
#reddit_posts['post_num_comments'].sort_values(ascending = False)

In [11]:
reddit_posts = reddit_posts.loc[reddit_posts['post_selftext'].notna(), :]

In [14]:
#reddit_posts['post_num_comments'].sort_values(ascending = False)

In [13]:
reddit_posts.loc[reddit_posts['post_num_comments'] > 100, ['post_id', 'post_num_comments']]

,post_id,post_num_comments
562,lwmpsz,124
1261,lwt425,109
1342,lwtmo0,393
2114,lx5fn9,129
3025,lxr5k6,270
3252,ly5fu3,236


In [7]:
reddit_sub = reddit_df.loc[reddit_df['post_id'] == 'lxr5k6', :]

In [8]:
reddit_sub.head()

,post_author,post_created_utc,post_domain,post_full_link,post_gildings,post_id,post_is_original_content,post_is_reddit_media_domain,post_locked,post_media_only,...,comment_is_submitter,comment_link_id,comment_parent_id,comment_permalink,comment_retrieved_on,comment_score,comment_stickied,comment_subreddit,comment_subreddit_id,comment_total_awards_received
3025,jona7946,1614881461,self.Denmark,https://www.reddit.com/r/Denmark/comments/lxr5...,{},lxr5k6,False,False,False,False,...,False,t3_lxr5k6,t3_lxr5k6,/r/Denmark/comments/lxr5k6/kan_vi_snakke_om_ga...,1615025529,32,False,Denmark,t5_2qjto,1
3026,jona7946,1614881461,self.Denmark,https://www.reddit.com/r/Denmark/comments/lxr5...,{},lxr5k6,False,False,False,False,...,True,t3_lxr5k6,t1_gpoh2me,/r/Denmark/comments/lxr5k6/kan_vi_snakke_om_ga...,1615025991,5,False,Denmark,t5_2qjto,0
3027,jona7946,1614881461,self.Denmark,https://www.reddit.com/r/Denmark/comments/lxr5...,{},lxr5k6,False,False,False,False,...,False,t3_lxr5k6,t3_lxr5k6,/r/Denmark/comments/lxr5k6/kan_vi_snakke_om_ga...,1615026285,227,False,Denmark,t5_2qjto,0
3028,jona7946,1614881461,self.Denmark,https://www.reddit.com/r/Denmark/comments/lxr5...,{},lxr5k6,False,False,False,False,...,False,t3_lxr5k6,t1_gpohyzk,/r/Denmark/comments/lxr5k6/kan_vi_snakke_om_ga...,1615026370,8,False,Denmark,t5_2qjto,0
3029,jona7946,1614881461,self.Denmark,https://www.reddit.com/r/Denmark/comments/lxr5...,{},lxr5k6,False,False,False,False,...,False,t3_lxr5k6,t3_lxr5k6,/r/Denmark/comments/lxr5k6/kan_vi_snakke_om_ga...,1615026486,305,False,Denmark,t5_2qjto,0


In [9]:
reddit_sub.shape

(91, 52)

In [10]:
reddit_sub.loc[3025, "post_selftext"]

'Den danske stat har aktier i Danske Spil og har Spilmyndigheden som ejer samtidig StopSpillet? Er det ikke lidt ligesom at sælge våben og bandager i krig? Man må ikke reklamere for cigaretter, men det er fint nok at der er 27 spil reklamer i hver reklame pause? Hvorfor er det okay at bombardere folk med spil og kviklån?'

In [15]:
# Definer tokenizer
def tokenizer_spacy(text):
    custom_stop_words = []
    stop_words = list(nlp.Defaults.stop_words) + custom_stop_words
    
    doc = nlp(text)
    
    pos_tags = ["NOUN", "ADJ", "PROPN"]
    
    tokens = []
    
    for word in doc:
        if len(word.text) < 5:
            continue
        if word.text.lower() not in stop_words and word.pos_ in pos_tags:
            tokens.append(word.lemma_.lower())
            
    return(tokens)

In [18]:
tokenizer_spacy("hej med dig din store ejendomsmæglere")

['stor', 'ejendomsmægler']

In [19]:
reddit_sub['tokens'] = reddit_sub['comment_body'].apply(tokenizer_spacy)

/tmp/ipykernel_927503/323066774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_sub['tokens'] = reddit_sub['comment_body'].apply(tokenizer_spacy)


In [20]:
reddit_sub_long = reddit_sub.explode('tokens')

In [21]:
reddit_sub_long['tokens'].value_counts()

reklamere      34
penge          11
reklame        10
betting        10
online          9
               ..
kryds           1
usandsynlig     1
sund            1
tilbud          1
overhånd        1
Name: tokens, Length: 534, dtype: int64

In [23]:
reddit_sub['emotion'] = reddit_sub['comment_body'].apply(classifier.predict)

/tmp/ipykernel_927503/3357303060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_sub['emotion'] = reddit_sub['comment_body'].apply(classifier.predict)


In [24]:
reddit_sub['emotion'].value_counts()

Foragt/Modvilje          41
Tillid/Accept            13
Forventning/Interrese    11
Glæde/Sindsro             7
Frygt/Bekymret            7
Overasket/Målløs          5
Sorg/trist                3
No emotion                3
Vrede/Irritation          1
Name: emotion, dtype: int64